# Projet Morpion
par *Hélène et Victoria*
![img](../../img/thymio.jpg)

Pour utiliser SenseHAT, il faut importer le module SensHAT pour avoir accès à la bibliothèque des fonctions liées à cet outil. IL faut donc créer une instance SenseHat().

In [19]:
from sense_emu import SenseHat
from time import sleep, time

sense = SenseHat()

/usr/lib/python3/dist-packages/sense_emu/sense_hat.py:91: Warning: No emulator detected; spawning sense_emu_gui
  warnings.warn(Warning('No emulator detected; spawning sense_emu_gui'))


Ensuite nous définissons les couleurs qui représentent le player 1 par exemple.

In [20]:
X = (255, 255, 255)
O = (0, 0, 0)
P1 = (0, 0, 255)
P2 = (255, 255, 0)
colors = (O, P1, P2)
colors

((0, 0, 0), (0, 0, 255), (255, 255, 0))

### Définition initiation 

Nous devons faire une défnition initation, où nous définissons l'état initial du jeu qui sera modifié par les joueurs.

Pour que nos arguments appelés *state*, *board*, *state_to_board* soit utilisables dans tous le code, il fat utiliser le mot-clé **global**.

In [21]:
def init():
    global state   
    global board  
    global state_to_board

Ensuite nous créeons la grille sur lequel a lieux ée jeu. 

In [22]:
board = [
        O, O, X, O, O, X, O, O,
        O, O, X, O, O, X, O, O,
        X, X, X, X, X, X, X, X,
        O, O, X, O, O, X, O, O,
        O, O, X, O, O, X, O, O,
        X, X, X, X, X, X, X, X,
        O, O, X, O, O, X, O, O,
        O, O, X, O, O, X, O, O, 
        ]
board[:3]

[(0, 0, 0), (0, 0, 0), (255, 255, 255)]

Nous créeons une matrice dont 4 pixels représente un élément de la matrice state 3x3, sans oublié de bien faire les sauts qui correspondent au cadrillage.

In [23]:
    state_to_board = [[(0, 1, 8, 9), (3, 4, 11, 12), (6, 7, 14, 15)],
                      [(24, 25, 32, 33), (27, 28, 35, 36), (30, 31, 38, 39)],
                      [(48, 49, 56, 57), (51, 52, 59, 60), (54, 55, 62, 63)]]

    state =  [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    state

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

### Définition Show_board(board, state)

Cette définitions nous permets d'afficher les différents états du jeu. Nous parcourons, les liste state et state_to_board pour aquérire à l'état de nos quatres pixels qui forment une case (soit les coordonées). Puis nous faisons correspondre son état, qui est 0,1,2 au position de la liste colors qui permettrons d'afficher l'état du jeu par exemple : jaune qui indique que c'est le player 2 qui a selectionnés la case.

In [25]:
def show_board(board, state):
    for y in range(len(state)):
        for x, s in enumerate(state[y]):  
            c = colors[s]
            for index in state_to_board[y][x]:
                board[index] = c
    sense.set_pixels(board)

In [26]:
show_board(board, state)

### Définition is_winning(p,state)

Nous définissons tous les cas ou un joueur est gagant. Pour cela il faut que l'état de trois cases selon les règles du jeu de morpion est le même état.

In [28]:
def is_winning(p, state):
    return  state[0][0] == state[0][1] == state[0][2] == p or \
            state[1][0] == state[1][1] == state[1][2] == p or \
            state[2][0] == state[2][1] == state[2][2] == p or \
            state[0][0] == state[1][0] == state[2][0] == p or \
            state[0][1] == state[1][1] == state[2][1] == p or \
            state[0][2] == state[1][2] == state[2][2] == p or \
            state[0][0] == state[1][1] == state[2][2] == p or \
            state[0][2] == state[1][1] == state[2][0] == p    

In [29]:
is_winning(1, state)

False

In [32]:
state =  [[1, 1, 1], [0, 0, 0], [0, 0, 0]]
show_board(board, state)
is_winning(2, state)

False

### Définitions is_draw

Si les cas de is_winnig(p,state) n'apparaissent pas dans le jeu, et que toutes les cases sont à l'état 1 ou 2 alors c'est un match nul. 

In [3]:
def is_draw(state): 
    for i in state:
        for s in i:
            if s == 0:
                return False
    return True

In [34]:
state = [[1,2,1],[1,2,2],[2,1,1]]
is_draw(state)

True

### Définition play(p, board, state)

Grâce à cette fonction nous définissons les paramètres du joueur.

D'abord nous définissons les paramètres du curseurs à l'état initial ((x,y) = (1,1)). Mais à l'état initial aucun joueur n'a encore joué donc la couleur du curseur est divisé par deux.Nous faisons cela en parcourant la liste state_to_board qui va donc prendre chaque etat des quatres pixels pour ensuite reparcourir la liste colors pour afficher l'état correspondant mais en divisant la couleur par deux. 

In [ ]:
def play(p,board, state):
    (x, y) = (1, 1) 
    dirs = {'up':(0, -1), 'down':(0, 1),
            'right':(1, 0), 'left':(-1, 0)} 
    
    c = tuple(int(x/2) for x in colors[p])
    for index in state_to_board[y][x]:
        board[index] = c
    sense.set_pixels(board)

Ensuite nous lançons une boucle qui attend un évènement de la bibliothèque directions pour modifier les coordonnées du curseur. Lorsqu'un évènment est préssé il ajoute la coordonées à celles initinales, mais les coordonées ne doivent pas etre plus "grandes" que notre matrice nous faisons donc un modulo.
Les coordonées sont donc modifiées mais pas forcément selectionnées pour de bon donc la case apparait encore d'une intensité de couleur diminuée.
```
    while True :
        event = sense.stick.wait_for_event()
        if event.action == 'pressed':
            if event.direction in dirs:  
                (dx, dy) = dirs[event.direction]
```

In [5]:
 
    while True :
        event = sense.stick.wait_for_event()
        if event.action == 'pressed':
            if event.direction in dirs:  
                (dx, dy) = dirs[event.direction]
                
                x = (x + dx) % len(state)    
                y = (y + dy) % len(state)
                
            
                           
                show_board(board, state)   # eviter de colorier le chemin
                
                c = tuple(int(x/2) for x in colors[p]) 
                for index in state_to_board[y][x]:
                    board[index] = c
                sense.set_pixels(board)   

NameError: name 'sense' is not defined

Il ne reste plus qu'a ajouter un paramètre : celui qui ne permet pas de modifier un état losque celui ci n'est pas égal à 0 .

In [7]:
           else:
                if state[y][x] == 0: 
                    state[y][x] = p
                    show_board(board, state)
                    return

SyntaxError: invalid syntax (<ipython-input-7-a63b7cd04862>, line 1)

### Définition end_game(p)

Lorsque aucun joueur gagne, on affiche le message *draw* autrement c'est le numéro du joueur gagant qui est affiché. Pour relancer la partie le message *press to play* apparait, il faut donc appuyer sur le joystick pour recommencer la partie.

In [37]:
                
def end_game(p):
    if p == 0:
        sense.show_message("draw")
    else:
        sense.show_letter(str(p))
        
    sleep(3)
    sense.show_message('press to play')
    
    while True:
        event = sense.stick.wait_for_event()
        if event.action == "pressed":   
            init()
            show_board(board, state)
            return
        

In [41]:
p=1
end_game(p)

### Le déroulement du jeu 

Nous lançons une boucle qui regarde si les cas gagnants ou match nul sont présents sinon la partie continue. Pour intervertir les joueurs entre 1 et 2 nous utilisons l'astuce **player = 3 - player**.

In [ ]:
init()
show_board(board, state)
player = 1

while True:
    play(player, board, state)
    if is_winning(player, state):
        end_game(player)
        
    elif is_draw(state):
        end_game(0)
        
    player = 3 - player    
